```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Plotting of Case B - Rossby Adjustment

This notebook is for creating informative figures for comparing different oceanographic properties between numerical schemes.
All data is expected to be read in from NetCDF files.

## This specific notebook intention:
Visualizing and plotting from the notebook from CaseB_RossbyAdjustment.ipynb

## How to use this notebook
This notebook is meaningless without existing result files. They can be obtained in two ways:
- Produce your own results by running the notebook CaseB_RossbyAdjustment.ipynb. Your results will be written to netCDF files in a folder containing a timestamp. In order to use those files in this notebook, **change the location of the result files in the cell that is reading data below**. This should be immediately under the heading "Read data and plot".
- Download the results that were presented in the paper from http://gpu-ocean.met.no:9000/python/netcdf_2018_01_24/, so that the default result file names in this notebook can be used.


#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec

import os
import pyopencl
import datetime
import sys

import scipy.io
import os.path

#Set large figure sizes
rc('figure', figsize=(5.0, 4.0))
rc('animation', html='html5')

#Import our Simulation Reader
from SWESimulators import Common, SimReader


### Define the different functions for evaluating some of the properties to be plotted.

In [ ]:
"""
Returns d/dx(0.5*g*h*h) + fhv
"""
def geostrophicBalanceEta(eta, H, hu, hv, nx, ny, dx, dy, f, g):
    return geostrophicBalance(eta+H, hu, hv, nx, ny, dx, dy, f, g)

def geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells in input arrays
    A = hu*hv/h
    B = hu*hu/h
    C = 0.5*g*h*h
    D = -f*hv
    
    Ay = np.zeros_like(A)
    Bx = np.zeros_like(B)
    Cx = np.zeros_like(C)
    
    Ay[1:-1,:] = (A[:-2,:] - A[2:,:])/(2*dy)
    Bx[:, 1:-1] = (B[:,:-2] - B[:,2:])/(2*dx)
    Cx[:, 1:-1] = (C[:,:-2] - C[:,2:])/(2*dx)
    
    #geoBalance = (Cx )/D - 1
    geoBalance = Cx - D
    #geoBalance = (Ay + Bx + Cx )/D - 1
    #geoBalance = (Ay + Bx + Cx - D)/(0.5*(Ay + Bx + Cx + D))
    return geoBalance

def geostrophicBalanceStaggered(eta, H, hu_s, hv_s, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
       
    h = eta + H
    hu = 0.5*(hu_s[:, :-1] + hu_s[:, 1:])
    hv = 0.5*(hv_s[:-1, :] + hv_s[1:, :])
    return geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g )


def kleinGordonBalance(eta, H, eta_0, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
    c = np.sqrt(g*H)
    a = c/f
    constant = 1/(a*a)
    print "a: , const: ", a, constant
    print "dx, dy:", dx, dy
    print "max eta-eta_0: ", np.max(eta - eta_0)
    
    eta_xx = np.zeros_like(eta)
    eta_yy = np.zeros_like(eta)
    laplace_eta = np.zeros_like(eta) 
        
    eta_yy[1:-1, :] = (eta[:-2,:] -2*eta[1:-1,:] + eta[2:, :])/(dy*dy)
    eta_xx[:, 1:-1] = (eta[:, :-2] - 2*eta[:, 1:-1] + eta[:, 2:])/(dx*dx)
    laplace_eta = eta_xx + eta_yy
    print "max Laplacean: ", np.max(laplace_eta)
    
    kg = laplace_eta - constant*(eta - eta_0)
    return kg
    
    

### Parameters used 
These should optimally be read from the NetCDF, but are currently just pasted in - in order to reuse existing plotting code

In [ ]:
#### Normal parameters

# Common parameters from Kai

nx = 800
ny = 1000

dx = 50000
dy = 50000

# dy*ny = 50 000 km. Dist(north_pole, south_pole) = 20 000 km
# dx*nx = 40 000 km = circumference of planEarth

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
H0 = 1000.0
A = 0.0 # A is diffusion coefficient multiplied by depth.

wind = Common.WindStressParams(type=99)

# Using an open boundary
sponge = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=sponge)

ghosts = [10, 10, 10, 10]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
validDomain =  [10, 10, 10, 10]

# The results are read without the ghost cells -> cutoff = 0
cutoff = 0 


# Rossby adjustment parameters:
c0_squared = g*H0
a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

#T = 300
#sub_T = 4000.0

T = 40
sub_T = 200*dt
make_netCDF = True
print "Tot sim time: ", T*sub_T
def get_sub_T(i):
    if i < 20:
        return sub_T
    else:
        return sub_T*10
    
interestingDomain = 250

paramSet = "Kai"
msg = "Using variables from " + paramSet

# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

# Domain for imshow
domain_extent = [ 0, dx*nx*1e-6, 0, dy*ny*1e-6 ]
xlabel = "$x$ ($10^6$ m)"
ylabel = "$y$ ($10^6$ m)"
labelfont = 13

## Read data and plot

In [ ]:
reload(SimReader)
folder = "netcdf_2018_01_24/"
fbl_file_name =   folder+"FBL_2018_01_24-15_24_57.nc"
ctcs_file_name =  folder+"CTCS_2018_01_24-17_10_12.nc"
kp07_file_name =  folder+"KP07_2018_01_24-17_46_27.nc" 
cdklm_file_name = folder+"CDKLM16_2018_01_24-17_15_16.nc"

imgdir='paper_images_rossby_adjustment_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
imgdir = imgdir + "/"

fbl_sim = SimReader.SimNetCDFReader(fbl_file_name)
ctcs_sim = SimReader.SimNetCDFReader(ctcs_file_name)
kp07_sim = SimReader.SimNetCDFReader(kp07_file_name)
cdklm_sim = SimReader.SimNetCDFReader(cdklm_file_name)


In [ ]:
cdklm_eta, cdklm_hu, cdklm_hv, t_end_cdklm = cdklm_sim.getLastTimeStep()
cdklm_eta0, t_start = cdklm_sim.getEtaAtTimeStep(0)
print "t_end_cdklm", t_end_cdklm

fig = plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)    
gb_CDKLM = geostrophicBalanceEta(cdklm_eta, \
                                 H0, \
                                 cdklm_hu, \
                                 cdklm_hv, \
                                 nx, ny, \
                                 dx, dy, \
                                 f, g)
plt.imshow(gb_CDKLM, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$x$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()

plt.subplot(1,2,2)
gb_CDKLM2 = geostrophicBalanceEta(cdklm_eta.T, \
                                  H0, \
                                  cdklm_hv.T, \
                                  cdklm_hu.T, \
                                  nx, ny, \
                                  dx, dy, \
                                  -f, g)
plt.imshow(gb_CDKLM2.T, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$y$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
plt.subplots_adjust(wspace=.3)
plt.suptitle("Geostrophic Balance CDKLM", fontsize=12)
fig.savefig(imgdir + "geoBalance_CDKLM_res.png")


fig = plt.figure(figsize=(5, 4))
KG_CDKLM = kleinGordonBalance(cdklm_eta, H0, \
                              cdklm_eta0, \
                              nx, ny, \
                              dx, dy, \
                              f, g)
plt.imshow(KG_CDKLM, origin='lower', extent=domain_extent)
plt.title("CDKLM - Klein-Gordon balance")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
fig.savefig(imgdir + "kleinGordon_CDKLM_res.png")

print imgdir

In [ ]:
ctcs_eta, ctcs_hu, ctcs_hv, t_end_ctcs = ctcs_sim.getLastTimeStep()
ctcs_eta0, t_start = ctcs_sim.getEtaAtTimeStep(0)
print "t_end_ctcs: ", t_end_ctcs

A = 0.0 # A is diffusion coefficient multiplied by depth.


fig = plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)    
gb_CTCS = geostrophicBalanceStaggered(ctcs_eta, H0, \
                               ctcs_hu, \
                               ctcs_hv, \
                               nx, ny, \
                               dx, dy, \
                               f, g)

plt.imshow(gb_CTCS, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$x$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()

plt.subplot(1,2,2)
gb_CTCS2 = geostrophicBalanceStaggered(ctcs_eta.T, H0, \
                                       ctcs_hv.T, \
                                       ctcs_hu.T, \
                                       nx, ny, \
                                       dx, dy, \
                                       -f, g)
plt.imshow(gb_CTCS2, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.colorbar()
plt.title("$y$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.subplots_adjust(wspace=.3)
plt.suptitle("Geostrophic Balance CTCS", fontsize=12)
fig.savefig(imgdir + "geoBalance_CTCS_res.png")


fig = plt.figure(figsize=(5, 4))
KG_CTCS = kleinGordonBalance(ctcs_eta, H0, \
                             ctcs_eta0, \
                             nx, ny, \
                             dx, dy, \
                             f, g)
plt.imshow(KG_CTCS, origin='lower', extent=domain_extent)
plt.title("CTCS - Klein-Gordon balance")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
fig.savefig(imgdir + "kleinGordon_CTCS_res.png")


In [ ]:
fbl_eta, fbl_hu, fbl_hv, t_end_fbl = fbl_sim.getLastTimeStep()
fbl_eta0, t_start = fbl_sim.getEtaAtTimeStep(0)
print "t_end_fbl: ", t_end_fbl

fig = plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)    
gb_FBL = geostrophicBalanceStaggered(fbl_eta, H0, \
                               fbl_hu, \
                               fbl_hv, \
                               nx, ny, \
                               dx, dy, \
                               f, g)

plt.imshow(gb_FBL, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$x$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()

plt.subplot(1,2,2)
gb_FBL2 = geostrophicBalanceStaggered(fbl_eta.T, H0, \
                                       fbl_hv.T, \
                                       fbl_hu.T, \
                                       nx, ny, \
                                       dx, dy, \
                                       -f, g)
plt.imshow(gb_FBL2, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$y$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
plt.subplots_adjust(wspace=.3)
plt.suptitle("Geostrophic Balance FBL", fontsize=12)
fig.savefig(imgdir + "geoBalance_FBL_res.png")


fig = plt.figure(figsize=(5, 4))
KG_FBL = kleinGordonBalance(fbl_eta, H0, \
                             fbl_eta0, \
                             nx, ny, \
                             dx, dy, \
                             f, g)
plt.imshow(KG_FBL, origin='lower', extent=domain_extent)
plt.title("FBL - Klein-Gordon balance")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
fig.savefig(imgdir + "kleinGordon_FBL_res.png")


In [ ]:
kp07_eta, kp07_hu, kp07_hv, t_end_kp07 = kp07_sim.getLastTimeStep()
kp07_eta0, t_start = kp07_sim.getEtaAtTimeStep(0)
print "t_end_kp07: ", t_end_kp07

fig = plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)    
gb_KP07 = geostrophicBalanceEta(kp07_eta, \
                                H0, \
                                kp07_hu, \
                                kp07_hv, \
                                nx, ny, \
                                dx, dy, \
                                f, g)
plt.imshow(gb_KP07, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$x$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()

plt.subplot(1,2,2)
gb_KP072 = geostrophicBalanceEta(kp07_eta.T, \
                                 H0, \
                                 kp07_hv.T, \
                                 kp07_hu.T, \
                                 nx, ny, \
                                 dx, dy, \
                                 -f, g)
plt.imshow(gb_KP072.T, interpolation="none", origin='lower', extent=domain_extent)#, vmin=-1, vmax =1)
plt.title("$y$-direction")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
plt.subplots_adjust(wspace=.3)
plt.suptitle("Geostrophic Balance KP07", fontsize=12)
fig.savefig(imgdir + "geoBalance_KP07_res.png")


fig = plt.figure(figsize=(5, 4))
#kp07_eta_0 = kp07_h0 - H0
KG_KP07 = kleinGordonBalance(kp07_eta, H0, \
                             kp07_eta0, \
                             nx, ny, \
                             dx, dy, \
                             f, g)
plt.imshow(KG_KP07, origin='lower', extent=domain_extent)
plt.title("KP07 - Klein-Gordon balance")
plt.xlabel(xlabel, fontsize=labelfont)
plt.ylabel(ylabel, fontsize=labelfont)
plt.colorbar()
fig.savefig(imgdir + "kleinGordon_KP07_res.png")


## Comparing final bump
Require the above cells to have been executed.

In [ ]:
## Read analytic solution
analytic_solution_file_name = "matlab/KG.mat"
analytic_solution_exists = False
if os.path.isfile(analytic_solution_file_name):
    analytic_solution = scipy.io.loadmat(analytic_solution_file_name)
    analytic_solution_exists = True
    print type(ctcs_eta0)
    print "------------------"
    print type(analytic_solution)
    for key, val in analytic_solution.iteritems():
        print key, ": ", type(val)
        if str(type(val)) == "<type 'numpy.ndarray'>":
            print "\t\t", val.shape
    print "-------------------------"
    
    # x (and y) is in the matlab file defined as  x=((1:Nx)-(Nx-1)/2)*dx
    # so we have to modified it according to the axis in our simulators
    
    analytic_eta      = analytic_solution['h']
    analytic_eta_init = analytic_solution['s']
    analytic_x        = analytic_solution['x'] # + (nx)*dx/2
    analytic_y        = analytic_solution['y'] # + (ny)*dy/2
    
    
    print cdklm_eta.shape
    print analytic_eta.shape, analytic_eta_init.shape, analytic_x.shape, analytic_y.shape
    
    xaxis, yaxis = cdklm_sim.getAxis()
    
    print "Comparing axis:"
    print xaxis.shape, yaxis.shape
    print xaxis[234], analytic_x[0,234], xaxis[234] - analytic_x[0,234], dx/2
    print yaxis[234], analytic_y[0,234], yaxis[234] - analytic_y[0,234]    
    print xaxis[0], dx
    
    fig = plt.figure(figsize=(5,5))
    plt.imshow(ctcs_eta0 - np.transpose(analytic_eta_init), origin="lower", extent=domain_extent)
    plt.colorbar()
    plt.xlabel(xlabel, fontsize=labelfont)
    plt.ylabel(ylabel, fontsize=labelfont)

    fig = plt.figure(figsize=(5,5))
    plt.imshow(kp07_eta0 - np.transpose(analytic_eta_init), origin="lower", extent=domain_extent)
    plt.colorbar()
    plt.xlabel(xlabel, fontsize=labelfont)
    plt.ylabel(ylabel, fontsize=labelfont)
    
    fig = plt.figure(figsize=(5,5))
    plt.imshow(ctcs_eta - np.transpose(analytic_eta), origin="lower", extent=domain_extent)
    plt.colorbar()
    plt.xlabel(xlabel, fontsize=labelfont)
    plt.ylabel(ylabel, fontsize=labelfont)
    
    fig = plt.figure(figsize=(5,5))
    plt.imshow(cdklm_eta - np.transpose(analytic_eta), origin="lower", extent=domain_extent)
    plt.colorbar()    
    plt.xlabel(xlabel, fontsize=labelfont)
    plt.ylabel(ylabel, fontsize=labelfont)


In [ ]:
# Comparison of bump across the different methods:
cutoff = 0
midx = (nx+2*cutoff)/2
midy = (ny+2*cutoff)/2

# From above:
#interestingDomain = 250
#print cutoff
interestingDomain = 100
xaxis, yaxis = cdklm_sim.getAxis()

print cutoff
print nx
print midx
print midx-interestingDomain
print midx+interestingDomain
print ctcs_eta0.shape
print ctcs_eta.shape, fbl_eta.shape, cdklm_eta.shape, kp07_eta.shape

fig = plt.figure(figsize=(10, 9))
plt.subplot(3,1,1)
plt.title("$\eta$ at geostrophic balance ($y=y_0$)")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], ctcs_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'k', label="Initial")
if (analytic_solution_exists):
    plt.plot(analytic_x[0,midx-interestingDomain:midx+interestingDomain], analytic_eta[midx-interestingDomain:midx+interestingDomain, midy], 'm', label="Reference")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], ctcs_eta[midy, midx-interestingDomain:midx+interestingDomain], 'b-.', label="CTCS")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], fbl_eta[midy, midx-interestingDomain:midx+interestingDomain], 'm:', label="FBL")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], cdklm_eta[midy, midx-interestingDomain:midx+interestingDomain], 'r--', label="CDKLM")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], kp07_eta[midy, midx-interestingDomain:midx+interestingDomain], 'g:', label="KP")
plt.xlim(xaxis[midx-interestingDomain], xaxis[midx+interestingDomain])
plt.ylim((-0.05, 0.25))
plt.legend()

#fig = plt.figure(figsize=(10, 9))
plt.subplot(3,1,2)
plt.title("$\eta$ at geostrophic balance ($x=x_0$)")
plt.plot(yaxis[midy-interestingDomain:midy+interestingDomain], ctcs_eta0[midy-interestingDomain:midy+interestingDomain, midx], 'k', label="Initial")
if (analytic_solution_exists):
    plt.plot(analytic_y[0, midy-interestingDomain:midy+interestingDomain], analytic_eta[midx, midy-interestingDomain:midy+interestingDomain], 'm', label="Reference")
plt.plot(yaxis[midy-interestingDomain:midy+interestingDomain], ctcs_eta[midy-interestingDomain:midy+interestingDomain, midx], 'b-.', label="CTCS")
plt.plot(yaxis[midy-interestingDomain:midy+interestingDomain], fbl_eta[midy-interestingDomain:midy+interestingDomain, midx], 'm:', label="FBL")
plt.plot(yaxis[midy-interestingDomain:midy+interestingDomain], cdklm_eta[midy-interestingDomain:midy+interestingDomain, midx], 'r--', label="CDKLM")
plt.plot(yaxis[midy-interestingDomain:midy+interestingDomain], kp07_eta[midy-interestingDomain:midy+interestingDomain, midx], 'g:', label="KP")
plt.xlim((yaxis[midy-interestingDomain], yaxis[midy+interestingDomain]))
plt.ylim((-0.05, 0.25))
plt.legend()

plt.subplot(3,1,3)
plt.title("$\eta$ at geostrophic balance - global ($y=y_0$) ")
plt.plot(xaxis, ctcs_eta0[midy, :], 'k', label="Init")
if (analytic_solution_exists):
    plt.plot(analytic_x[0,:], analytic_eta[:, midy], 'm', label="Analytic")
plt.plot(xaxis, ctcs_eta[midy, :], 'b-.', label="CTCS")
plt.plot(xaxis, fbl_eta[midy, :], 'm:', label="FBL")
plt.plot(xaxis, cdklm_eta[midy, :], 'r--', label="CDKLM")
plt.plot(xaxis, kp07_eta[midy, :], 'g:', label="KP07")
plt.ylim((-0.05, 0.25))
plt.legend()

fig.subplots_adjust(hspace=.3)
fig.savefig(imgdir + "rossby_adjustment_x_and_y.png")


# Compare initial conditions in order to see that we plot the same regions
fig = plt.figure(figsize=(5, 4))
plt.title("$\eta$ Initial conditions ($y=y_0$)")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], ctcs_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'b-.', label="CTCS")
if (analytic_solution_exists):
    plt.plot(analytic_x[0,midx-interestingDomain:midx+interestingDomain], analytic_eta_init[midx-interestingDomain:midx+interestingDomain, midy], 'm', label="Reference")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], fbl_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'm:', label="FBL")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], cdklm_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'r--', label="CDKLM")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain], kp07_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'g:', label="KP")
plt.xlim(xaxis[midx-interestingDomain], xaxis[midx+interestingDomain])
plt.ylim((-0.05, 0.25))
plt.grid()
plt.legend()
fig.savefig(imgdir + "rossby_adjustment_inits.png")


print t_end_fbl, t_end_ctcs, t_end_kp07, t_end_cdklm
print np.max(ctcs_eta0), np.max(fbl_eta0), np.max(cdklm_eta0), np.max(kp07_eta0)
print np.max(ctcs_eta), np.max(fbl_eta), np.max(cdklm_eta), np.max(kp07_eta)
print (np.max(fbl_eta)  /np.max(fbl_eta0))
print (np.max(ctcs_eta) /np.max(ctcs_eta0))
print (np.max(kp07_eta) /np.max(kp07_eta0))
print (np.max(cdklm_eta)/np.max(cdklm_eta0))


fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(6,1)
ax0 = plt.subplot2grid((6,1), (0,0), rowspan=2)
plt.title("$\eta$ at geostrophic balance ($y=y_0$)", fontsize=labelfont)
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, ctcs_eta0[midy, midx-interestingDomain:midx+interestingDomain], 'k', label="Initial")
if (analytic_solution_exists):
    plt.plot(analytic_x[0,midx-interestingDomain:midx+interestingDomain]*1e-3, analytic_eta[midx-interestingDomain:midx+interestingDomain, midy], 'm', label="Reference")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, ctcs_eta[midy, midx-interestingDomain:midx+interestingDomain], 'b-.', label="CTCS")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, fbl_eta[midy, midx-interestingDomain:midx+interestingDomain], 'm:', label="FBL")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, cdklm_eta[midy, midx-interestingDomain:midx+interestingDomain], 'r--', label="CDKLM")
plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, kp07_eta[midy, midx-interestingDomain:midx+interestingDomain], 'g:', label="KP")
plt.xlim(xaxis[midx-interestingDomain]*1e-3, xaxis[midx+interestingDomain]*1e-3)
plt.ylim((-0.05, 0.3))
plt.xlabel("$x$ ($10^3$ m)", fontsize=labelfont)
plt.ylabel("$\eta$ (m)", fontsize=labelfont)
plt.grid()
plt.legend()

#ax0 = plt.subplot(gs[4])
fsize = 6
ax0 = plt.subplot2grid((6,1), (2,0))
fbl_sim.makeInfoPlot(ax0, text_font_size=fsize)
#ax0 = plt.subplot(gs[5])
ax0 = plt.subplot2grid((6,1), (3,0))
ctcs_sim.makeInfoPlot(ax0, text_font_size=fsize)
#ax0 = plt.subplot(gs[6])
ax0 = plt.subplot2grid((6,1), (4,0))
kp07_sim.makeInfoPlot(ax0, text_font_size=fsize)
#ax0 = plt.subplot(gs[7])
ax0 = plt.subplot2grid((6,1), (5,0))
cdklm_sim.makeInfoPlot(ax0, text_font_size=fsize)

fig.subplots_adjust(hspace=.6)
fig.savefig(imgdir + "rossby_adjustment_results.png")


### PLOT Residuals
if (analytic_solution_exists):
    
    fbl_residual   =   fbl_eta[midy, midx-interestingDomain:midx+interestingDomain] - analytic_eta[midx-interestingDomain:midx+interestingDomain, midy]
    ctcs_residual  =  ctcs_eta[midy, midx-interestingDomain:midx+interestingDomain] - analytic_eta[midx-interestingDomain:midx+interestingDomain, midy]
    kp07_residual  =  kp07_eta[midy, midx-interestingDomain:midx+interestingDomain] - analytic_eta[midx-interestingDomain:midx+interestingDomain, midy]
    cdklm_residual = cdklm_eta[midy, midx-interestingDomain:midx+interestingDomain] - analytic_eta[midx-interestingDomain:midx+interestingDomain, midy]

    print "Result shape  : ", ctcs_eta0[midy, midx-interestingDomain:midx+interestingDomain].shape
    print "Reference shape: ", analytic_eta[midx-interestingDomain:midx+interestingDomain, midy].shape
    print "Residual shape: ", ctcs_residual.shape
    #print "Residual      : ", ctcs_residual
    
    maxSteadyState = np.max(analytic_eta)
    
    
    fig = plt.figure(figsize=(10, 12))
    gs = gridspec.GridSpec(8,1)
    ax0 = plt.subplot2grid((8,1), (0,0), rowspan=2)
    plt.title("$\eta_{rel} = (\eta-\eta_{ref})/\max(\eta_{ref})$ at geostrophic balance ($y=y_0$)")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, ctcs_residual/maxSteadyState, 'b-.', label="CTCS")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, fbl_residual/maxSteadyState, 'm:', label="FBL")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, cdklm_residual/maxSteadyState, 'r--', label="CDKLM")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, kp07_residual/maxSteadyState, 'g:', label="KP")
    plt.xlim(xaxis[midx-interestingDomain]*1e-3, xaxis[midx+interestingDomain]*1e-3)
    #plt.ylim((-0.05, 0.25))
    plt.grid()
    plt.xlabel("$x$ ($10^3$ m)", fontsize=labelfont)
    plt.ylabel("$\eta_{rel}$", fontsize=labelfont)
    plt.legend(loc=4)
    
    #ax0 = plt.subplot(gs[4])
    fsize = 6
    
    ax0 = plt.subplot2grid((8,1), (2,0), rowspan=2)
    plt.title("$\eta_{rel} = (\eta-\eta_{ref})/\max(\eta_{ref})$ at geostrophic balance ($y = y_0$)")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, ctcs_residual/maxSteadyState, 'b-.', label="CTCS")
    plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, fbl_residual/maxSteadyState, 'm:', label="FBL")
    #plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, cdklm_residual, 'r--', label="CDKLM")
    #plt.plot(xaxis[midx-interestingDomain:midx+interestingDomain]*1e-3, kp07_residual, 'g:', label="KP07")
    plt.xlim(xaxis[midx-interestingDomain]*1e-3, xaxis[midx+interestingDomain]*1e-3)
    #plt.ylim((-0.05, 0.25))
    plt.grid()
    plt.xlabel("$x$ ($10^3$ m)", fontsize=labelfont)
    plt.ylabel("$\eta_{rel}$", fontsize=labelfont)
    plt.legend(loc=4)
    
    ax0 = plt.subplot2grid((8,1), (4,0))
    fbl_sim.makeInfoPlot(ax0, text_font_size=fsize)
    #ax0 = plt.subplot(gs[5])
    ax0 = plt.subplot2grid((8,1), (5,0))
    ctcs_sim.makeInfoPlot(ax0, text_font_size=fsize)
    #ax0 = plt.subplot(gs[6])
    ax0 = plt.subplot2grid((8,1), (6,0))
    kp07_sim.makeInfoPlot(ax0, text_font_size=fsize)
    #ax0 = plt.subplot(gs[7])
    ax0 = plt.subplot2grid((8,1), (7,0))
    cdklm_sim.makeInfoPlot(ax0, text_font_size=fsize)

    fig.subplots_adjust(hspace=1.5)
    fig.savefig(imgdir + "rossby_adjustment_residuals.png")



In [ ]:
## Investigate radial symmetry


y_coords, x_coords = np.mgrid[dy/2.0:(ny+0.5)*dy:dy, dx/2.0:(nx+0.5)*dx:dx]
x_center = dx*nx/2.0
y_center = dy*ny/2.0

print x_coords.shape
print y_coords.shape
print y_coords[:,4].shape
print "should be zero: ", (y_coords[:, 346] - yaxis)[5]
print "should be zero: ", (x_coords[346, :] - xaxis)[5]

x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
min_x = np.min(x_coords[:,0]);
min_y = np.min(y_coords[0,:]);

max_x = np.max(x_coords[0,:]);
max_y = np.max(y_coords[:,0]);
print "{min|max}{x|y}:",  min_x, max_x, min_y, max_y

def plotRadialSymmetry(ax, eta, leg):
    sp_radial1, = ax.plot(radius.ravel()*1e-6, eta.ravel(), '.')#, label=leg)
    sp_radial1, = ax.plot(xaxis[0:nx/2]*1e-6, eta[ny/2, nx/2:], '.y')
    invisible = ax.plot([], [], 'w', label=leg)
    #plt.axis([0, min(max_x, max_y), 59.96, 60.08])
    #plt.title(title)
    plt.xlabel("Radius from center ($10^6$ m)", fontsize=labelfont)
    plt.ylabel("$\eta$ (m)", fontsize=labelfont)
    plt.xlim(0, 7.5)
    plt.legend(loc=1, frameon=False)
    
    
allSims = [fbl_sim, ctcs_sim, kp07_sim, cdklm_sim]
allEtas = [fbl_eta, ctcs_eta, kp07_eta, cdklm_eta]
#allTitles = ["FBL radial symmetry", "CTCS radial symmetry", "KP radial symmetry", "CDKLM radial symmetry"]
allLegends = ["FBL", "CTCS", "KP", "CDKLM"]
fig = plt.figure(figsize=(10, 22)) 
gs = gridspec.GridSpec(8,2)
for i in range(4):
    ax0 = plt.subplot(gs[i])
    plotRadialSymmetry(ax0, allEtas[i], allLegends[i])
    ax1 = plt.subplot(gs[4 + i*2])    
    allSims[i].makeInfoPlot(ax1, text_font_size=10)

fig.tight_layout()
fig.subplots_adjust(hspace=.5)
fig.savefig(imgdir + "rossby_adjustment_radial_symmetry.png")


In [ ]:
print imgdir
